In [1]:
import pandas as pd
import os
absolutepath = os.path.abspath(os.getcwd())
fileDirectory = os.path.dirname(absolutepath)
parentDirectory = os.path.dirname(fileDirectory)
dataprocDirectory = os.path.join(fileDirectory, 'data/proc')
datarawDirectory = os.path.join(fileDirectory, 'data/raw')

In [2]:
balanced_dataset = pd.read_csv(os.path.join(dataprocDirectory,
                                                   'balanced_panel_wo_affiliation_never_treated_control_group.csv'))

In [3]:
balanced_dataset['relative_period'] = balanced_dataset['year'] - balanced_dataset['cohort']

balanced_dataset = balanced_dataset[balanced_dataset.relative_period>-7]

In [4]:
import numpy as np
main_outcomes = ['num_docs','frac_count_docs','if_top10','if_top50','if_lower50',
                 'sum_citescore','funded']
for o in main_outcomes:
    balanced_dataset[f"ihs_{o}"] = np.arcsinh(balanced_dataset[o])*100

In [5]:
balanced_dataset['ever_treated'] = 0
balanced_dataset.loc[(balanced_dataset.treatment_status==1),'ever_treated'] = 1
relative_periods = balanced_dataset.relative_period.unique()
# create time period X treatment vars
# all periods
for rp in relative_periods:
    balanced_dataset[f"relative_period{rp}Xtreated"] = balanced_dataset.ever_treated*\
    (balanced_dataset.relative_period==rp)

balanced_dataset["postXtreated"] = balanced_dataset.ever_treated*\
                                          (balanced_dataset.relative_period>=0)

balanced_dataset["post0to3Xtreated"] = balanced_dataset.ever_treated*\
                                            ((balanced_dataset.relative_period>=0)&\
                                             (balanced_dataset.relative_period<4))
balanced_dataset["post4to8Xtreated"] = balanced_dataset.ever_treated*\
                                             (balanced_dataset.relative_period>=4)

In [6]:
cohort_af = pd.read_csv((os.path.join(dataprocDirectory,
                    'author_cohort_jttp_af.csv')))

In [7]:
balanced_dataset = balanced_dataset.merge(cohort_af, how = 'left', on =['authid','cohort'])

In [8]:
balanced_dataset.count()

authid                              12251921
cohort                              12251921
year                                12251921
subfield_most_frequent              12251921
subfield_most_frequent_two_digit    12251921
                                      ...   
postXtreated                        12251921
post0to3Xtreated                    12251921
post4to8Xtreated                    12251921
afid_masked_unique_filtered         11246003
afid_masked_nunique_filtered        11246003
Length: 78, dtype: int64

# drop only joiners

In [9]:
balanced_dataset = balanced_dataset[balanced_dataset.only_joiner!=1]

In [10]:
balanced_dataset.shape

(9117833, 78)

In [11]:
balanced_dataset.to_csv(os.path.join(dataprocDirectory,'balanced_panel_never_treated_control_group.csv'),index=False)